## Imports & display options

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from IPython.display import display

# Optional: nicer DataFrame display
pd.set_option('display.float_format', '{:,.4f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Config & data loading

In [2]:
file_path = "activity-data.csv"
target_column = 'activity'
indexes_to_remove = []  # rows to drop (empty = keep all)

# Read CSV
df = pd.read_csv(file_path)

# Drop specified rows
df_cleaned = df.drop(index=indexes_to_remove)


## Target/feature split & basic cleaning

In [18]:
# Target
Y = df_cleaned[target_column]

# Features: drop target and identifier (if present)
X = df_cleaned.drop(columns=[target_column, 'Folder_Name'], errors='ignore')

# Fill NaNs with column means, then drop any columns still containing NaNs
X_filled = X.fillna(X.mean(numeric_only=True))
X_filled = X_filled.dropna(axis=1)

# Remove near-constant features
X_cleaned = X_filled.loc[:, X_filled.var() > 0.01]

X_cleaned.shape

(1018, 1391)

## Correlation pruning

In [19]:
corr = X_cleaned.corr(method='pearson').abs()
upper = np.triu(np.ones(corr.shape), k=1)

high_i, high_j = np.where((corr.values > 0.85) & (upper == 1))
drop_features = []
cols = corr.columns

for i, j in zip(high_i, high_j):
    fi, fj = cols[i], cols[j]
    if 'buried_volume' in fj:
        drop_features.append(fi)
    elif 'buried_volume' in fi:
        drop_features.append(fj)
    elif '_bol' in fj:
        drop_features.append(fi)
    elif '_bol' in fi:
        drop_features.append(fj)
    else:
        drop_features.append(fj)

drop_features = list(set(drop_features))
X_reduced = X_cleaned.drop(columns=drop_features, errors='ignore')

X_reduced.shape

(1018, 183)

## Scaling & PCA-derived top-20 features

In [20]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_reduced)

# PCA to retain 95% variance; compute weighted-squared-loadings importance
pca = PCA(n_components=0.95)
pca.fit(X_scaled)
loadings = pca.components_
explained = pca.explained_variance_ratio_
feature_importance = np.sum(explained.reshape(-1, 1) * loadings**2, axis=0)

# Sort and take top-20
sorted_idx = np.argsort(feature_importance)[::-1]
important_features = X_reduced.columns[sorted_idx]
top_20_features_PCA = important_features[:20]
X_PCA_top_20 = X_reduced[top_20_features_PCA]

list(top_20_features_PCA)

['6_Q_buried_volume_ave_scomformer',
 'Alpha_polarizability_bol',
 'MinEStateIndex',
 'time',
 '15_G4_bol',
 '4_G3_bol',
 'SlogP_VSA10',
 '1_G4_bol',
 'NumRotatableBonds',
 'SPS',
 'MinAbsEStateIndex',
 'R2_G3_bol',
 '6_Q_buried_volume_2_scomformer',
 'Distance_1_8_bol',
 '4_G4_bol',
 'PEOE_VSA6',
 'R2_G4_bol',
 '3_Q_buried_volume_min_scomformer',
 '15_G3_bol',
 '3_O_buried_volume_max_bol']

## Lasso- and RF-based feature selection

In [21]:
# LassoCV selection
lasso = LassoCV(cv=5, max_iter=10000, random_state=42)
lasso.fit(X_scaled, Y)
lasso_abs = np.abs(lasso.coef_)
top_20_features_lasso = X_reduced.columns[np.argsort(lasso_abs)[::-1][:20]]
X_LASSO_top_20 = X_reduced[top_20_features_lasso]

# Random Forest importance
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_reduced, Y)
rf_imp = rf.feature_importances_
top_20_features_rf = X_reduced.columns[np.argsort(rf_imp)[::-1][:20]]
X_RF_top_20 = X_reduced[top_20_features_rf]

# Combined (Lasso ∪ RF), deduplicated
X_combined = pd.concat([X_RF_top_20, X_LASSO_top_20], axis=1)
X_combined = X_combined.loc[:, ~X_combined.columns.duplicated()]

X_LASSO_top_20.shape, X_RF_top_20.shape, X_combined.shape

((1018, 20), (1018, 20), (1018, 32))

## Train/test split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_combined, Y, test_size=0.2, random_state=42)

# Refit scaler on the training slice for fair evaluation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled.shape, X_test_scaled.shape

((814, 32), (204, 32))

## Define regressors

In [23]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import ElasticNet, Lasso
from xgboost import XGBRegressor

models = {
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42),
    'RF': RandomForestRegressor(
        n_estimators=300, max_depth=6, min_samples_split=2, 
        min_samples_leaf=1, bootstrap=True, random_state=42
    ),
    'AdaBoost': AdaBoostRegressor(
        n_estimators=100, learning_rate=0.5, random_state=42
    ),
    'XGBoost': XGBRegressor(
        n_estimators=100, max_depth=6, learning_rate=0.2,
        subsample=0.8, colsample_bytree=0.9, random_state=42
    ),
    'LASSO': Lasso(alpha=0.1, random_state=42)
}

list(models.keys())

['Linear Regression',
 'Ridge Regression',
 'Random Forest',
 'Support Vector Regression',
 'XGBoost',
 'Lasso Regression']

## Train, cross-validate, and evaluate

In [24]:
results = []

for model_name, model in models.items():
    # 5-fold CV on training set (neg MSE -> MSE)
    cv_scores = cross_val_score(
        model, X_train_scaled, y_train, cv=5,
        scoring='neg_mean_squared_error', n_jobs=-1
    )
    mean_cv_mse = -cv_scores.mean()

    # Fit
    model.fit(X_train_scaled, y_train)

    # Predict
    y_tr_pred = model.predict(X_train_scaled)
    y_te_pred = model.predict(X_test_scaled)

    # Metrics
    mse_train = mean_squared_error(y_train, y_tr_pred)
    mse_test  = mean_squared_error(y_test,  y_te_pred)
    r2_train  = r2_score(y_train, y_tr_pred)
    r2_test   = r2_score(y_test,  y_te_pred)
    mae_train = mean_absolute_error(y_train, y_tr_pred)
    mae_test  = mean_absolute_error(y_test,  y_te_pred)

    results.append([
        model_name, mse_train, mse_test, r2_train, r2_test, mae_train, mae_test, mean_cv_mse
    ])

results_df = pd.DataFrame(results, columns=[
    'Model', 'Train MSE', 'Test MSE', 'Train R²', 'Test R²', 'Train MAE', 'Test MAE', 'CV MSE'
])

display(results_df)

Model               Train MSE                Test MSE  \
0          Linear Regression 27,671,566,007,709.2383 25,985,948,482,260.2148   
1           Ridge Regression 27,671,569,686,293.0234 25,984,741,712,933.1211   
2              Random Forest  8,362,821,403,472.7744 16,674,496,290,491.9531   
3  Support Vector Regression 49,179,643,523,306.8516 43,888,515,188,521.0469   
4                    XGBoost  2,948,986,493,001.6377 15,051,429,517,089.7949   
5           Lasso Regression 27,671,566,007,712.2695 25,985,947,554,798.8125   

   Train R²  Test R²      Train MAE       Test MAE                  CV MSE  
0    0.3928   0.3568 3,548,425.7624 3,661,005.6627 30,657,931,938,234.3633  
1    0.3928   0.3568 3,548,417.2993 3,660,870.2430 30,655,183,013,240.4258  
2    0.8165   0.5873 2,142,477.5551 2,876,611.3329 14,976,639,598,856.6250  
3   -0.0791  -0.0864 4,323,406.2907 4,195,404.3935 49,454,521,642,908.4219  
4    0.9353   0.6274 1,208,118.9312 2,607,262.4954 15,335,789,529,458.1621  
5    0.3928   0.3568 3,548,425.7905 3,661,005.5658 30,657,930,911,254.6992